In [1]:
import torch
from torch.nn import init

class MyCNN(torch.nn.Module):
    def __init__(self):
        super(MyCNN, self).__init__()
        self.conv1 = torch.nn.Conv2d(3, 6, 5)
        self.conv2 = torch.nn.Conv2d(6, 16, 5)
        self.fc1 = torch.nn.Linear(16 * 5 * 5, 120)
        self.fc2 = torch.nn.Linear(120, 84)
        self.fc3 = torch.nn.Linear(84, 10)

        self.apply(my_custom_weight_init)

    def forward(self, x):
        x = self.conv1(x)
        x = torch.nn.functional.relu(x)
        x = self.conv2(x)
        x = torch.nn.functional.relu(x)
        x = x.view(-1, 16 * 5 * 5)
        x = self.fc1(x)
        x = torch.nn.functional.relu(x)
        x = self.fc2(x)
        x = torch.nn.functional.relu(x)
        x = self.fc3(x)
        return x

def my_custom_weight_init(tensor):
    if isinstance(tensor, torch.nn.Linear):
        torch.nn.init.orthogonal_(tensor.weight.data)
        tensor.bias.data.fill_(1.0)
    elif isinstance(tensor, torch.nn.Conv2d):
        tensor.weight.data.fill_(1.0)
        tensor.bias.data.fill_(1.0)

model = MyCNN()
#model.apply(my_custom_weight_init)

for name, param in model.named_parameters():
    #print(name, param.data.size())
    print(name, param)


conv1.weight Parameter containing:
tensor([[[[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]],

         [[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]],

         [[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]]],


        [[[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]],

         [[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.]],

         [[1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1., 1.],
          [1., 1., 1., 1.

In [2]:
a = [1, 2, 3]
print(a[::-1])

[3, 2, 1]


## Custom Initialization of Lazy Linear Layer

In [17]:
import torch 
import torch.nn as nn 
import numpy as np

def get_output_shape(model, image_dim):
    return model(torch.rand(*(image_dim))).data.shape

class Encoder(nn.Module):
    def __init__(self,
    obs_shape,
    feature_dim,
    conv_layers = [64, 32, 32],
    dense_layers = [1028, 512, 256]) -> None:
        super().__init__()
        assert len(obs_shape) == 3, "image observation of shape (c, w, h) is expected"
        self.obs_shape = obs_shape
        self.feature_dim = feature_dim
        self.conv_layers = conv_layers 
        self.dense_layers = dense_layers 
        self.conv_out_shape = None 


        self.convs = nn.Sequential()
        for i in range(len(self.conv_layers)):
            if i == 0:
                self.convs.append(nn.Conv2d(self.obs_shape[0], self.conv_layers[i], 
                                                 kernel_size=3, stride=1))
            else:
                self.convs.append(nn.Conv2d(self.conv_layers[i-1], self.conv_layers[i],
                                                 kernel_size=3, stride=1))
            self.convs.append(nn.ReLU())
            self.convs.append(nn.MaxPool2d(kernel_size=2))

        
        conv_out = get_output_shape(self.convs, self.obs_shape)
        fc_input = np.prod(list(conv_out))
        print('Conv output size: ', conv_out)
        print('fc_input size:', fc_input)

        self.fcs = nn.Sequential()
        for i in range(len(self.dense_layers)):
            if i == 0:
                #self.fcs.append(nn.LazyLinear(self.dense_layers[i]))
                self.fcs.append(nn.Linear(fc_input, self.dense_layers[i]))
            else:
                self.fcs.append(nn.Linear(self.dense_layers[i-1], self.dense_layers[i]))
            self.fcs.append(nn.ReLU())
        self.fcs.append(nn.Linear(self.dense_layers[-1], self.feature_dim))

        
    def forward(self, obs, detach=False):
        obs = obs / 255.0

        out = self.convs(obs)
        out = torch.flatten(out, start_dim=1)  # flatten starting from dim = 1
        out = self.fcs(out)

        if detach:
            out = out.detach()
        return out

obs_shape = (3, 100, 100)  # (C, H, W)
enc = Encoder(obs_shape, feature_dim=10)

input_img = torch.rand(5, 3, 100, 100)
print('input image size:', input_img.size())
output = enc(input_img)

print(output.size())

Conv output size:  torch.Size([32, 10, 10])
fc_input size: 3200
input image size: torch.Size([5, 3, 100, 100])
torch.Size([5, 10])


## Auto Encoder 

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(32, 16, 2, stride=2),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 3, 2, stride=2),
            nn.Sigmoid(),
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


x = torch.rand((1, 3, 32, 32))
autoenc = Autoencoder()
x_recon = autoenc(x) 

print(x_recon.size())


torch.Size([1, 3, 32, 32])


In [10]:
def assert_match_tuples(tuple1, tuple2):
    """Assert that two tuples match.

    Args:
        tuple1 (tuple): The first tuple.
        tuple2 (tuple): The second tuple.

    Raises:
        AssertionError: If the two tuples do not match.
    """

    assert len(tuple1) == len(tuple2), "The two tuples must have the same length."
    for i in range(len(tuple1)):
        assert tuple1[i] == tuple2[i], "The two tuples do not match at index {}.".format(i)

if __name__ == "__main__":
    tuple1 = (1, 2, 3)
    tuple2 = (1, 2, 3)
    assert_match_tuples(tuple1, tuple2)

    tuple1 = (1, 2, 3)
    tuple2 = (1, 2, 4)
    try:
        assert_match_tuples(tuple1, tuple2)
    except AssertionError as e:
        print(e)


The two tuples do not match at index 2.
